In [ ]:
import pandas as pd

In [ ]:
cases = pd.read_csv("V4_3.csv")#pd.read_csv("Validated_V0_Desriptions.csv")

In [ ]:
data = pd.read_csv("data/raw_data_new_.csv", parse_dates=["block_time"])

In [ ]:
data.head()

In [ ]:
#data['amount_usd']
#examinations of data integrity, check if data is corrupt from current load
#data[(data['amount_usd'].isnull()) & (data['total_balance_usd'] ==0 ) & (data['current_token_balance'] ==0 ) & (data['current_token_to_usd'] ==0 ) & (data['current_token_balance_usd'] ==0 )].to_csv('empty_amount_usd.csv')
			

In [ ]:
df = data.groupby(['file_name'])['label'].sum().reset_index(name = 'n_attacks')  
df.loc[df['n_attacks'] == 1, 'sngl_mlt'] = 'single'
df.loc[df['n_attacks'] > 1, 'sngl_mlt'] = 'multi'
df = df[['file_name' , 'sngl_mlt']]

In [ ]:
data.columns

In [ ]:
#case_split = cases.loc[:, ["File Names", "train/test splits"]]
#case_split.columns = ["file_name", "experiment_part"]
#case_split["file_name"] += ".csv"


case_split = cases.loc[:, ["File Names", "train/test splits"]]#cases.loc[:, ["File Names", "train/test splits"]]
case_split.columns = ["file_name", "experiment_part"]
case_split["file_name"] += ".csv"
case_split["file_name"] = 'Balance_V4/' + case_split['file_name'].astype(str)#case_split["file_name"] = 'V3/' + case_split['file_name'].astype(str)

In [ ]:
data.shape

In [ ]:
data = data.merge(case_split)

In [ ]:
data.shape

In [ ]:
###  data = data.merge(df)  #### <<--- df = multi/single

In [ ]:
data.shape

In [ ]:
data["experiment_part"].value_counts()

In [ ]:
#data['sngl_mlt'].value_counts()

In [ ]:
data

## Configure featrure engineering

In [ ]:
from cyvers_ai_ds.features import AssetCentricFEPipeline
from cyvers_ai_ds.features.transaction import DEFAULT_AGG_LIST

In [ ]:
pipe = AssetCentricFEPipeline()

In [ ]:
out_data = pipe.execute_(data)

In [ ]:
tx_df, asset_df = out_data

In [ ]:
#tx_df['amount_usd']#
#tx_df['amount_usd_bal']#tx_df['current_token_balance_usd'] 

In [ ]:
tx_df.shape

In [ ]:
tx_df.columns

In [ ]:
tx_metadata = data.loc[
    :, ["transaction_id", "file_name", "experiment_part"]#, "sngl_mlt"]
].drop_duplicates()

In [ ]:
tx_metadata.shape

In [ ]:
tx_df = tx_df.merge(tx_metadata)

In [ ]:
tx_df.shape

In [ ]:
tx_df["experiment_part"].value_counts()


In [ ]:
### check how rations differ for fraud/non-fraud transactions per file
df1 = tx_df[tx_df['label'] == 1][['file_name' , 'amount_usd_bal']].groupby(['file_name']).mean().reset_index()#['amount_usd_bal'].mean() , tx_df[tx_df['label'] == 0]['amount_usd_bal'].mean()
df0 = tx_df[tx_df['label'] == 0][['file_name' , 'amount_usd_bal']].groupby(['file_name']).mean().reset_index()

df1.rename(columns={'amount_usd_bal': 'fraud'}, inplace=True); df0.rename(columns={'amount_usd_bal': 'nfraud'}, inplace=True)
df_r = df1.merge(df0)#.to_csv('balance_ratios.csv')#df1 , df0
print(df_r)
#df_r.to_csv('balance_ratios_.csv')

In [ ]:
###overall ratios for fraud/non-fraud trx
tx_df[tx_df['label'] == 1]['amount_usd_bal'].mean() , tx_df[tx_df['label'] == 0]['amount_usd_bal'].mean()

In [ ]:
#tx_df["sngl_mlt"].value_counts()


In [ ]:
tx_df['file_name'].unique().shape

In [ ]:
tx_df.loc[tx_df["experiment_part"] == "Train", :].to_csv(
    "data/train_data_b1.csv", index=False
)
tx_df.loc[tx_df["experiment_part"] == "Validation", :].to_csv(
    "data/validation_data_b1.csv", index=False
)
tx_df.loc[tx_df["experiment_part"] == "Test", :].to_csv(
    "data/test_data_b1.csv", index=False
)

Back up data to s3

In [ ]:
from utils import MODEL_NAME
from utils.s3 import MODELS_BUCKET
import boto3

In [ ]:
#don't push the data as of yet
bucket = boto3.resource('s3').Bucket(MODELS_BUCKET)
bucket.upload_file('data/train_data.csv', MODEL_NAME + '/data/train_data.csv')
bucket.upload_file('data/validation_data.csv', MODEL_NAME + '/data/validation_data.csv')
bucket.upload_file('data/test_data.csv', MODEL_NAME + '/data/test_data.csv')